In [39]:
# Loading in all of my necessary libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import pyarrow.parquet as pq
pd.set_option('display.max_columns', None)

In [3]:
# Reading in the Taxi Zones Lookup table and taking a look at it
Taxi_Zones = pd.read_csv("taxi_zone_lookup.csv")
Taxi_Zones

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NaN,NaN


In [4]:
# Reading in my Yellow Taxi Cab dataset and inspecting its shape
Taxi_Data = pd.read_parquet("yellow_tripdata_2020-01.parquet")
print(Taxi_Data.shape)

(6405008, 19)


In [5]:
# Build lookup dictionaries
borough_lookup = Taxi_Zones.set_index("LocationID")["Borough"].to_dict()
zone_lookup    = Taxi_Zones.set_index("LocationID")["Zone"].to_dict()

# Map directly into Taxi_Data
Taxi_Data["PU_Borough"] = Taxi_Data["PULocationID"].map(borough_lookup)
Taxi_Data["PU_Zone"]    = Taxi_Data["PULocationID"].map(zone_lookup)
Taxi_Data["DO_Borough"] = Taxi_Data["DOLocationID"].map(borough_lookup)
Taxi_Data["DO_Zone"]    = Taxi_Data["DOLocationID"].map(zone_lookup)

# Optional: reorder so new cols sit after IDs
cols = list(Taxi_Data.columns)
for loc, boro, zone in [
    ("PULocationID", "PU_Borough", "PU_Zone"),
    ("DOLocationID", "DO_Borough", "DO_Zone")
]:
    idx = cols.index(loc)
    cols = cols[:idx+1] + [boro, zone] + [c for c in cols[idx+1:] if c not in [boro, zone]]

Taxi_Data = Taxi_Data[cols]

In [6]:
# Taking a Look at the dataset to see if my columpns updated correctly
Taxi_Data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,PU_Borough,PU_Zone,DOLocationID,DO_Borough,DO_Zone,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,Manhattan,Upper West Side North,239,Manhattan,Upper West Side South,1,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,None
1,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,Manhattan,Upper West Side South,238,Manhattan,Upper West Side North,1,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5,None
2,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,Manhattan,Upper West Side North,238,Manhattan,Upper West Side North,1,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5,None
3,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,Manhattan,Upper West Side North,151,Manhattan,Manhattan Valley,1,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0,None
4,2,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,Queens,Queensbridge/Ravenswood,193,Queens,Queensbridge/Ravenswood,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0,None


In [12]:
# Count of the null values seen in each column of the dataset
Taxi_Data.isnull().sum()

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count                0
trip_distance                  0
RatecodeID                     0
store_and_fwd_flag             0
PULocationID                   0
PU_Borough                  3090
PU_Zone                    43779
DOLocationID                   0
DO_Borough                 14858
DO_Zone                    39678
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge           0
airport_fee              6339567
dtype: int64

In [8]:
# Count rows with Y vs N in store_and_fwd_flag
print(Taxi_Data['store_and_fwd_flag'].value_counts(dropna=False))

store_and_fwd_flag
N       6271447
Y         68120
None      65441
Name: count, dtype: int64


In [9]:
# Creating a df of all the rows with missing values in its "store_and_fwd_column"
missing_fwd = Taxi_Data[Taxi_Data['store_and_fwd_flag'].isna()]

# Count how many of these also have passenger_count or RatecodeID missing
print("Total rows with store_and_fwd_flag NaN:", missing_fwd.shape[0])
print("... with passenger_count NaN:", missing_fwd['passenger_count'].isna().sum())
print("... with RatecodeID NaN:", missing_fwd['RatecodeID'].isna().sum())

# Rows where all 3 are missing
all_three_missing = missing_fwd[
    missing_fwd['passenger_count'].isna() & missing_fwd['RatecodeID'].isna()
]
print("Rows with all three missing:", all_three_missing.shape[0])
# Drop rows where passenger_count, RatecodeID, and store_and_fwd_flag are all NaN
Taxi_Data = Taxi_Data.dropna(subset=['passenger_count', 'RatecodeID', 'store_and_fwd_flag'])

Total rows with store_and_fwd_flag NaN: 65441
... with passenger_count NaN: 65441
... with RatecodeID NaN: 65441
Rows with all three missing: 65441


In [23]:
# Identifying that 264 and 265 are special Location ID's as 264's Borough is "unknown" and its zone is "NaN"
# 265's borough is NaN and its zone is "Outside of NYC"
special_ids = [264, 265]

summary = []

# This creates a table showing how many rows have 264 or 265 as its DO or PU Location ID, and
# the count of missing values in these rows respective PU and DO Borough and Zones
for loc_id in special_ids:
    # PU stats
    pu_rows = Taxi_Data[Taxi_Data['PULocationID'] == loc_id]
    pu_total = len(pu_rows)
    pu_missing_borough = pu_rows['PU_Borough'].isna().sum()
    pu_missing_zone = pu_rows['PU_Zone'].isna().sum()
    
    # DO stats
    do_rows = Taxi_Data[Taxi_Data['DOLocationID'] == loc_id]
    do_total = len(do_rows)
    do_missing_borough = do_rows['DO_Borough'].isna().sum()
    do_missing_zone = do_rows['DO_Zone'].isna().sum()
    
    summary.append({
        "LocationID": loc_id,
        "PU_Count": pu_total,
        "PU_Borough_NaN": pu_missing_borough,
        "PU_Zone_NaN": pu_missing_zone,
        "DO_Count": do_total,
        "DO_Borough_NaN": do_missing_borough,
        "DO_Zone_NaN": do_missing_zone
    })

summary_df = pd.DataFrame(summary)
summary_df


,LocationID,PU_Count,PU_Borough_NaN,PU_Zone_NaN,DO_Count,DO_Borough_NaN,DO_Zone_NaN
0,264,43779,0,43779,39678,0,39678
1,265,3090,3090,0,14858,14858,0


In [37]:
Taxi_Data = Taxi_Data[
    (Taxi_Data['PULocationID'] != 264) & (Taxi_Data['DOLocationID'] != 264)
]
count_264 = Taxi_Data[
    (Taxi_Data['PULocationID'] == 264) | (Taxi_Data['DOLocationID'] == 264)
].shape[0]
print("Rows with PU or DO LocationID = 264:", count_264)
print("The New Shape of the dataset is now: ", Taxi_Data.shape)

Rows with PU or DO LocationID = 264: 0
The New Shape of the dataset is now:  (6284005, 23)


In [30]:
# Rows where passenger_count == 0
zero_passengers = Taxi_Data[Taxi_Data['passenger_count'] == 0]

# Number of rows with passenger_count == 0
print("Rows with passenger_count == 0:", zero_passengers.shape[0])
# Count NaNs per column for rows where passenger_count == 0
na_counts_zero_passengers = zero_passengers.isna().sum()

print(na_counts_zero_passengers)

Rows with passenger_count == 0: 113085
VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count               0
trip_distance                 0
RatecodeID                    0
store_and_fwd_flag            0
PULocationID                  0
PU_Borough                   83
PU_Zone                       0
DOLocationID                  0
DO_Borough                  298
DO_Zone                       0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge          0
airport_fee              113085
dtype: int64


In [31]:
# Rows in which the pickup time was later than the Dropoff Time
# Keep only rows where dropoff is after pickup

# Count rows where dropoff is before pickup
invalid_time_rows = (Taxi_Data['tpep_dropoff_datetime'] < Taxi_Data['tpep_pickup_datetime']).sum()

print("Number of rows with dropoff before pickup:", invalid_time_rows)

Taxi_Data = Taxi_Data[
    Taxi_Data['tpep_dropoff_datetime'] >= Taxi_Data['tpep_pickup_datetime']
]

Number of rows with dropoff before pickup: 0


In [32]:
# Count negative trip distances
negative_count = (Taxi_Data['trip_distance'] < 0).sum()
print("Number of trips with negative distance:", negative_count)
# Filter for negative trip distances
#negative_trips = Taxi_Data[Taxi_Data['trip_distance'] < 0]
#negative_trips
#Taxi_Data = Taxi_Data[Taxi_Data['trip_distance'] >= 0]

Number of trips with negative distance: 13


In [33]:
# Count trips with distance > 100 miles
extra_long_count = (Taxi_Data['trip_distance'] > 100).sum()
extra_long_trips = Taxi_Data[Taxi_Data['trip_distance'] > 100]
print("Number of trips with distance > 100 miles:", extra_long_count)


Number of trips with distance > 100 miles: 20


In [34]:
negative_fares = Taxi_Data[Taxi_Data['fare_amount'] < 0]
# Number of rows with negative fares
print("Rows with negative fares:", negative_fares.shape[0])

Rows with negative fares: 18800


In [35]:
high_fares_count = (Taxi_Data['fare_amount'] > 500).sum()
print("Number of trips with fare_amount > 500:", high_fares_count)
high_fares = Taxi_Data[Taxi_Data['fare_amount'] > 500]

Number of trips with fare_amount > 500: 15


In [36]:
# Filter rows where passenger_count > 6
too_many_passengers = Taxi_Data[Taxi_Data['passenger_count'] > 6]

# Number of rows
print("Rows with passenger_count > 6:", too_many_passengers.shape[0])

Rows with passenger_count > 6: 45
